In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import mlflow
import plotly.graph_objects as go
import plotly.express as px
import mlflow.sklearn
import mlflow.xgboost
import xgboost as xgb
import optuna
import os
import json

from dotenv import load_dotenv
from sklearn.model_selection import train_test_split
from scipy.stats import norm
from sksurv.metrics import concordance_index_censored

##### Test GPU

In [ ]:
# Test GPU par XGBoost
try:
    # On crée une micro-matrice de test
    data = xgb.DMatrix([[1, 2], [3, 4]], label=[1, 0])

    params = {'tree_method': 'gpu_hist', 'device': 'cuda'}
    xgb.train(params, data, num_boost_round=1)
    print("✅ Succès ! La RTX 4060 est reconnue et configurée.")
except Exception as e:
    print(f"❌ Échec du GPU : {e}")
    print("Le modèle tournera sur CPU par défaut.")

In [ ]:
df = pd.read_parquet('dataset_full.parquet')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
pd.set_option('display.float_format', '{:.4f}'.format)

print(f"Structure du dataset : {df.shape[0]} lignes et {df.shape[1]} colonnes")
display(df.head())

In [ ]:
# --- AJUSTEMENTS POUR LE MODÈLE SURVIVAL AFT ---

# 1. NETTOYAGE GÉOGRAPHIQUE
df['Code du département de l\'établissement'] = df['Code du département de l\'établissement'].astype(str).str.zfill(2)
dep_risk_map = df.groupby("Code du département de l'établissement")["fermeture"].mean()
df['risque_departemental'] = df['Code du département de l\'établissement'].map(dep_risk_map)

# 2. TRAITEMENT DES TYPES
df['Catégorie juridique de l\'unité légale'] = df['Catégorie juridique de l\'unité légale'].astype(str)
df['age_estime'] = df['age_estime'].astype(float)
df['Tranche_effectif_num'] = df['Tranche_effectif_num'].fillna(0).astype(float)

# 3. ENCODAGE DES VARIABLES

df['is_ess'] = df['Economie sociale et solidaire unité légale'].map({'O': 1, 'N': 0}).fillna(0).astype(int)

# B. One-Hot Encoding

df_final = pd.get_dummies(
    df, 
    columns=['libelle_section_ape', 'Catégorie juridique de l\'unité légale'], 
    prefix=['APE', 'CJ'],
    drop_first=True,
    dtype=int 
)

# 4. SÉLECTION FINALE

cols_to_drop = [
    'Code postal de l\'établissement', 'Code commune de l\'établissement',
    'Activité principale de l\'unité légale', 'Date_fermeture_finale', 
    'latitude', 'longitude', 'code_ape',
    'Code du département de l\'établissement', 'Code de la région de l\'établissement',
    'Economie sociale et solidaire unité légale'
]
df_final = df_final.drop(columns=[c for c in cols_to_drop if c in df_final.columns])

# 5. DERNIERS RÉGLAGES POUR LA SURVIE (Crucial pour AFT)

df_final = df_final[df_final['age_estime'] > 0].copy()

# On crée les colonnes cibles pour le modèle AFT

df_final['y_lower'] = df_final['age_estime']
df_final['y_upper'] = np.where(df_final['fermeture'] == 1, df_final['age_estime'], np.inf)

print(f"✅ Dataset finalisé : {df_final.shape[0]} lignes, {df_final.shape[1]} colonnes.")

In [ ]:
# --- 1. ANALYSE DES FRÉQUENCES ---
binary_cols = [c for c in df_final.columns if c.startswith('APE_') or c.startswith('CJ_')]
frequencies = df_final[binary_cols].mean().sort_values(ascending=False) * 100

# Définition du seuil (0.1%)
rare_limit = 0.1
rare_cols = frequencies[frequencies < rare_limit]

print(f"--- 🔍 Analyse des colonnes rares (< {rare_limit}%) ---")
print(f"Il y a {len(rare_cols)} colonnes concernées.")

# --- 2. FUSION DES CATÉGORIES RARES ---

rare_ape_cols = [c for c in rare_cols.index if c.startswith('APE_')]
rare_cj_cols = [c for c in rare_cols.index if c.startswith('CJ_')]

# On crée la colonne "Autres" et on supprime les anciennes
if rare_ape_cols:
    df_final['APE_Autres_Secteurs'] = df_final[rare_ape_cols].any(axis=1).astype(int)
    df_final.drop(columns=rare_ape_cols, inplace=True)

if rare_cj_cols:
    df_final['CJ_Autres_Status'] = df_final[rare_cj_cols].any(axis=1).astype(int)
    df_final.drop(columns=rare_cj_cols, inplace=True)

# --- 3. PRÉPARATION DES CIBLES DE SURVIE (AFT) ---

df_final['y_lower'] = df_final['age_estime']
df_final['y_upper'] = np.where(df_final['fermeture'] == 1, df_final['age_estime'], np.inf)

print(f"✅ Nettoyage et préparation AFT terminés.")
print(f"📊 Nouveau nombre de colonnes : {len(df_final.columns)}")
display(df_final[['age_estime', 'fermeture', 'y_lower', 'y_upper']].head())

##### Train/test split

In [ ]:
# 1. On s'assure d'avoir la colonne d'âge
if 'age_au_diagnostic' not in df_final.columns:
    df_final['age_au_diagnostic'] = df_final['age_estime']

# 2. Définition des colonnes à exclure (on garde l'âge !)
# Assure-toi que 'risque_departemental' est BIEN DANS to_drop si tu veux l'enlever
non_numeric_cols = df_final.select_dtypes(exclude=[np.number]).columns.tolist()
targets = ['fermeture', 'age_estime', 'y_lower', 'y_upper']
to_drop = list(set(non_numeric_cols + targets))

if 'age_au_diagnostic' in to_drop:
    to_drop.remove('age_au_diagnostic')

# --- OPTIONNEL : Si tu veux enlever le risque départemental pour rééquilibrer ---

X = df_final.drop(columns=to_drop)
y_time = df_final['age_estime']
y_event = df_final['fermeture'].astype(int)

# 3. Split Train/Test
X_train, X_test, y_train_time, y_test_time, y_train_event, y_test_event = train_test_split(
    X, y_time, y_event, test_size=0.2, random_state=42
)

# 4. Création des DMatrix (Le cerveau d'XGBoost)
# Pour le train
y_upper_train = np.where(y_train_event == 1, y_train_time, np.inf)
dtrain = xgb.DMatrix(X_train)
dtrain.set_float_info('label_lower_bound', y_train_time.values)
dtrain.set_float_info('label_upper_bound', y_upper_train)

# Pour le test (Optuna en a besoin pour évaluer)
y_upper_test = np.where(y_test_event == 1, y_test_time, np.inf)
dtest_optuna = xgb.DMatrix(X_test)
dtest_optuna.set_float_info('label_lower_bound', y_test_time.values)
dtest_optuna.set_float_info('label_upper_bound', y_upper_test)

print(f"✅ Données prêtes ! Features utilisées : {X.columns.tolist()}")

##### Train dans Mlflow

In [ ]:
# # --- 1. GESTION DE L'EXPERIMENT MLFLOW ---

# experiment_name = "XGBoost_Survival_V3_Equilibre"

# try:

#     exp_id = mlflow.create_experiment(experiment_name)
# except:

#     exp_id = mlflow.get_experiment_by_name(experiment_name).experiment_id

# mlflow.set_experiment(experiment_name)

# # --- 2. DÉFINITION DE L'OBJECTIF ---
# def objective(trial):
#     params = {
#         'objective': 'survival:aft',
#         'eval_metric': 'aft-nloglik',
#         'tree_method': 'hist',
#         'device': 'cuda',
        
#         # --- RÉGLAGES POUR LA NUANCE (Anti-Binaire) ---
#         'max_depth': trial.suggest_int('max_depth', 3, 6), 
#         'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.02, log=True),
        
#         # Force des feuilles plus denses pour éviter le 0% de risque brutal
#         'min_child_weight': trial.suggest_int('min_child_weight', 10, 50),
        
#         # Régularisation forte pour contrer la domination du département
#         'lambda': trial.suggest_float('lambda', 1.0, 50.0, log=True),
#         'alpha': trial.suggest_float('alpha', 0.1, 10.0, log=True),
        
#         'aft_loss_distribution': 'logistic', 
#         'aft_loss_distribution_scale': trial.suggest_float('aft_loss_distribution_scale', 0.8, 1.5),
        
#         # On réduit le sampling pour que chaque arbre voit des sous-ensembles différents
#         'subsample': trial.suggest_float('subsample', 0.5, 0.8),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.8),
#     }

#     # Un run "nested" (imbriqué) pour chaque essai Optuna
#     with mlflow.start_run(nested=True):
#         bst_trial = xgb.train(
#             params, 
#             dtrain, 
#             num_boost_round=2000, 
#             evals=[(dtest_optuna, 'test')],
#             early_stopping_rounds=100,
#             verbose_eval=False
#         )
        
#         score = bst_trial.best_score
#         mlflow.log_params(params)
#         mlflow.log_metric("test_nloglik", score)
        
#         return score

# # --- 3. LANCEMENT DE L'OPTIMISATION ---
# study = optuna.create_study(direction='minimize')

# # On englobe l'étude dans un run parent identifié par l'exp_id
# with mlflow.start_run(run_name="Train_V3_Equilibre_Anciennete", experiment_id=exp_id):
#     study.optimize(objective, n_trials=50)

# # --- 4. RÉCUPÉRATION DU MEILLEUR MODÈLE ---
# print(f"✅ Meilleur score : {study.best_value}")
# print(f"🏆 Paramètres : {study.best_params}")

---

##### Sauvegarde du modèle

In [ ]:
# # 1. Récupération des meilleurs paramètres
# best_params = study.best_params
# best_params.update({
#     'objective': 'survival:aft',
#     'eval_metric': 'aft-nloglik',
#     'tree_method': 'hist',
#     'device': 'cuda',
#     'aft_loss_distribution': 'logistic'
# })

# print("🚀 Ré-entraînement du modèle final avec les meilleurs paramètres...")

# # 2. Entraînement final (on utilise dtrain qui contient 100% du train)
# final_model = xgb.train(
#     best_params,
#     dtrain,
#     num_boost_round=2000 # On peut monter un peu plus pour la version finale
# )

# # 3. Sauvegarde au format JSON (plus sûr pour ton API)
# final_model.save_model("model_survie_V3_final.json")
# print("✅ Modèle V3 sauvegardé sous 'model_survie_V3_final.json'")

---

##### Vérification des features importances sur les données de test

In [ ]:
# --- 1. PERFORMANCE SUR LE TEST SET ---
# On calcule le score final sur les données de test non vues
eval_result = final_model.eval(dtest_optuna)
print(f"📊 Performance finale (Test NLogLik) : {eval_result}")

# --- 2. IMPORTANCE DES VARIABLES ---
# On regarde ce qui a vraiment compté pour le modèle
importance = final_model.get_score(importance_type='gain')
importance_df = pd.DataFrame({
    'Feature': importance.keys(),
    'Importance': importance.values()
}).sort_values(by='Importance', ascending=False)

print("\n🏆 Top 10 des variables les plus influentes :")
print(importance_df.head(10))

---

#### Test de prédictions

In [ ]:
# --- 3. PRÉDICTION ET RÉPARTITION ---

preds_mu = final_model.predict(dtest_optuna)
sigma = best_params['aft_loss_distribution_scale']

# On calcule un risque arbitraire à 2 ans
def calculate_risk(mu, horizon=2, s=sigma):
    z = (np.log(horizon) - mu) / s
    return (1 / (1 + np.exp(-z))) * 100

# Application aux résultats
risques_2ans = [calculate_risk(m) for m in preds_mu]

# Création d'un mini-df d'analyse
analysis_df = pd.DataFrame({
    'mu': preds_mu,
    'risk_2y': risques_2ans,
    'age_reel': X_test['age_au_diagnostic']
})

# Définition des profils (ex: > 50% de risque à 2 ans = Critique)
critique = analysis_df[analysis_df['risk_2y'] > 50]
solide = analysis_df[analysis_df['risk_2y'] <= 10]

print("-" * 50)
print(f"🏢 Entreprises analysées (Test) : {len(analysis_df)}")
print(f"🔴 Portefeuille critique (>50% risque) : {len(critique)} ({len(critique)/len(analysis_df):.1%})")
print(f"🟢 Portefeuille solide (<10% risque) : {len(solide)} ({len(solide)/len(analysis_df):.1%})")
print("-" * 50)
print(f"💡 Âge moyen (Profil Critique) : {critique['age_reel'].mean():.1f} ans")
print(f"👴 Âge moyen (Profil Solide) : {solide['age_reel'].mean():.1f} ans")

In [ ]:
def calculate_risk_safe(mu, horizon=5, s=sigma):
    # On utilise np.clip pour éviter que z ne devienne trop grand (overflow)
    z = (np.log(horizon) - mu) / s
    z = np.clip(z, -50, 50) 
    # Logistique robuste
    return (1 / (1 + np.exp(-z))) * 100

# On teste sur un horizon plus lointain (5 ans) pour voir les fragilités
analysis_df['risk_5y'] = analysis_df['mu'].apply(lambda x: calculate_risk_safe(x, horizon=5))

# On baisse un peu le seuil du "critique" pour le diagnostic (ex: 20% à 5 ans)
critique = analysis_df[analysis_df['risk_5y'] > 20]
solide = analysis_df[analysis_df['risk_5y'] <= 5]

print("-" * 50)
print(f"🏢 Entreprises analysées : {len(analysis_df)}")
print(f"🔴 Profil Fragile (>20% risque à 5 ans) : {len(critique)} ({len(critique)/len(analysis_df):.1%})")
print(f"🟢 Profil Très Solide (<5% risque à 5 ans) : {len(solide)} ({len(solide)/len(analysis_df):.1%})")
print("-" * 50)
if len(critique) > 0:
    print(f"💡 Âge moyen (Fragile) : {critique['age_reel'].mean():.1f} ans")
print(f"👴 Âge moyen (Solide) : {solide['age_reel'].mean():.1f} ans")

##### Vérification des risques du portefeuille

In [ ]:
# 1. Préparation de la matrice globale (X_total)

X_total = df_final[X.columns] 
dmatrix_total = xgb.DMatrix(X_total)

# 2. Prédiction des Mu et calcul des risques
preds_mu_total = final_model.predict(dmatrix_total)
sigma = best_params['aft_loss_distribution_scale']

def get_risk(mu, horizon=5):
    z = (np.log(horizon) - mu) / sigma
    z = np.clip(z, -50, 50)
    return (1 / (1 + np.exp(-z))) * 100

# 3. Création du DataFrame de répartition
ptf_analysis = pd.DataFrame({
    'age': df_final['age_au_diagnostic'],
    'risk_5y': [get_risk(m) for m in preds_mu_total]
})

# 4. Segmentation en échelons (Labels de risque)
bins = [0, 5, 10, 20, 100]
labels = ['🟢 Très Solide', '🟡 Stable', '🟠 Fragile', '🔴 Critique']
ptf_analysis['segment'] = pd.cut(ptf_analysis['risk_5y'], bins=bins, labels=labels)

# 5. Calcul de la répartition
repartition = ptf_analysis.groupby('segment', observed=True).agg(
    nombre_entreprises=('age', 'count'),
    age_moyen=('age', 'mean')
).reset_index()

repartition['pourcentage'] = (repartition['nombre_entreprises'] / len(ptf_analysis) * 100).round(2)

print("📊 RÉPARTITION GLOBALE DU PORTEFEUILLE (Horizon 5 ans)")
print("-" * 60)
print(repartition.to_string(index=False))
print("-" * 60)

##### Vérification des secteurs les plus à risques

In [ ]:
# Top 5 des secteurs les plus représentés dans le segment Critique
critique_df = df_final.loc[ptf_analysis[ptf_analysis['segment'] == '🔴 Critique'].index]
top_ape_critique = critique_df.filter(like='APE_').sum().sort_values(ascending=False).head(5)

print("🔥 Top 5 des secteurs les plus à risque (Segment Critique) :")
print(top_ape_critique)

---